In [ ]:
import pandas as pd

df = pd.read_csv('dataset.tsv', names=['size', 'width', 'height', 'data'], sep='\t')
df.head()

,size,width,height,data
0,a,60,60,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,a,60,60,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
2,a,60,60,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
3,a,60,60,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
4,a,60,60,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [ ]:
# TensorFlow と tf.keras のインポート
import tensorflow as tf
from tensorflow import keras

# ヘルパーライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt

import base64
from sklearn.model_selection import train_test_split

In [ ]:
class_names = sorted(df['size'].unique().tolist())
weight = {i: 1 - (j / len(df)) for i, j in df['size'].value_counts().items()}
weights = {idx: weight[c] for idx, c in enumerate(class_names)}

df['size'] = [class_names.index(x) for x in df['size']]
df['data'] = [base64.b64decode(d.encode()) for d in df['data']]
DATASET_SIZE = len(df)

In [ ]:

df_train, df_test = train_test_split(df, test_size=0.1, random_state=10101, stratify=df['size'])
df_train, df_valid = train_test_split(df_train, test_size=0.1, random_state=10101, stratify=df_train['size'])

train_source = df_train.pop('data')
train_target = df_train.pop('size')
train_dataset = tf.data.Dataset.from_tensor_slices((train_source, train_target.values))

test_source = df_test.pop('data')
test_target = df_test.pop('size')
test_dataset = tf.data.Dataset.from_tensor_slices((test_source, test_target.values))

valid_source = df_valid.pop('data')
valid_target = df_valid.pop('size')
valid_dataset = tf.data.Dataset.from_tensor_slices((valid_source, valid_target.values))

In [ ]:
def decode_data(data, target):
  image = tf.image.decode_jpeg(data, channels=3)
  image = tf.image.resize(image, [60, 60])
  image /= 255.0  # normalize to [0,1] range
  return image, target

In [ ]:
train_dataset = train_dataset.map(decode_data, num_parallel_calls=tf.data.AUTOTUNE).shuffle(10).batch(12)
test_dataset = test_dataset.map(decode_data, num_parallel_calls=tf.data.AUTOTUNE).shuffle(10).batch(12)
valid_dataset = valid_dataset.map(decode_data, num_parallel_calls=tf.data.AUTOTUNE).shuffle(10).batch(12)

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.4,fill_mode = 'reflect'),
  tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor=0.2, fill_mode =  'reflect'),
  tf.keras.layers.experimental.preprocessing.RandomTranslation(height_factor=0.2, width_factor=0.2, fill_mode="reflect"),
  tf.keras.layers.experimental.preprocessing.RandomContrast(factor=0.2),
  tf.keras.layers.experimental.preprocessing.RandomHeight(factor=0.2),
  tf.keras.layers.experimental.preprocessing.RandomWidth(factor=0.2)
])

In [ ]:
train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x, training=True), y), 
                num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
efn2 = tf.keras.applications.EfficientNetV2B0(input_shape=(60, 60, 3), include_top=False)

model = tf.keras.Sequential([
    efn2,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(len(class_names))])

model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-b0 (Function  (None, 2, 2, 1280)       5919312   
 al)                                                             
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 8)                 10248     
                                                                 
Total params: 5,929,560
Trainable params: 5,868,952
Non-trainable params: 60,608
_________________________________________________________________


In [ ]:
epochs=50
history = model.fit(
  train_dataset,
  validation_data=valid_dataset,
  epochs=epochs,
  class_weight=weights
)

Epoch 1/50
70/70 [==============================] - 27s 208ms/step - loss: 7.6094 - accuracy: 0.1166 - val_loss: 9.5646 - val_accuracy: 0.0860
Epoch 2/50
70/70 [==============================] - 8s 113ms/step - loss: 7.4864 - accuracy: 0.1382 - val_loss: 9.3980 - val_accuracy: 0.0753
Epoch 3/50
70/70 [==============================] - 6s 87ms/step - loss: 7.3509 - accuracy: 0.1370 - val_loss: 6.0393 - val_accuracy: 0.1398
Epoch 4/50
70/70 [==============================] - 5s 75ms/step - loss: 7.1559 - accuracy: 0.1575 - val_loss: 9.3882 - val_accuracy: 0.1935
Epoch 5/50
70/70 [==============================] - 5s 72ms/step - loss: 7.0673 - accuracy: 0.1502 - val_loss: 12.4752 - val_accuracy: 0.0323
Epoch 6/50
70/70 [==============================] - 5s 66ms/step - loss: 6.8182 - accuracy: 0.1851 - val_loss: 12.0277 - val_accuracy: 0.0323
Epoch 7/50
70/70 [==============================] - 4s 60ms/step - loss: 6.9859 - accuracy: 0.1683 - val_loss: 9.6479 - val_accuracy: 0.0645
Epoch 8/